In [1]:
import networkx as nx
import time
from scipy.optimize import minimize
from qiskit import QuantumCircuit, Aer
from qiskit.circuit import Parameter
import numpy
from joblib import Parallel, delayed
import itertools

In [2]:
def graph_partitioning(G, max_node = 30, cluster_limit = 15):
    #max_node = len(G.nodes)
    check_nodes = [False for i in range(0, max_node)]
    G_remaining = G.copy()
    list_new = []
    G_new = nx.DiGraph()

    while True:
        check_all = True
        for node in G.nodes:
            if not check_nodes[node]:
                check_all = False
                break
            if check_all:
                break

        max_degree = -1
        selected_node = -1
        for node in G_remaining:
            G_remaining.nodes[node]['score'] = 0
            if G_remaining.degree[node] > max_degree:
                max_degree = G_remaining.degree[node]
                selected_node = node
        if selected_node == -1:
            break
        G_new = nx.DiGraph()
        G_new.add_node(selected_node)

        check_bfs = [False for i in range(0, max_node)]
        for node in G_new.nodes:
            check_bfs[node] = True

        check_bfs[selected_node] = True
        queue_bfs = [selected_node]
        disjoint_check = True
        while len(queue_bfs) != 0 and len(G_new.edges) <= cluster_limit:
            head_node = queue_bfs[0]
            total_score = 0
            for node in G_remaining.neighbors(head_node):
                if not check_bfs[node]:
                    G_remaining.nodes[node]['score'] += 1
            for node in G_remaining.predecessors(head_node):
                if not check_bfs[node]:
                    G_remaining.nodes[node]['score'] += 1

            disjoint_check = True
            for node in G_remaining.nodes:
                if check_bfs[node]:
                    continue
                nums_edge = 0
                for node_new in G_new.nodes:
                    if G_remaining.has_edge(node, node_new) or G_remaining.has_edge(node_new, node):
                        nums_edge += 1
                if nums_edge + len(G_new.edges) > cluster_limit:
                    #print(G_new.edges, nums_edge, node, cluster_limit)
                    G_remaining.nodes[node]['score'] = 0
                    check_bfs[node] = True

            for node in G_remaining.nodes:
                total_score += G_remaining.nodes[node]['score']

            if total_score == 0:
                break

            candidate = []
            weight = []
            for node in G_remaining.nodes:
                if G_remaining.nodes[node]['score'] > 0:
                    candidate.append(node)
                    weight.append(G_remaining.nodes[node]['score']/total_score)

            selected_node = numpy.random.choice(numpy.array(candidate), p=weight)
            #print(candidate, selected_node)
            new_edges = []
            for node_new in G_new.nodes:
                if G_remaining.has_edge(selected_node, node_new):
                    new_edges.append((selected_node, node_new))
                if G_remaining.has_edge(node_new, selected_node):
                    new_edges.append((node_new, selected_node))
            G_new.add_edges_from(new_edges)

            G_remaining.nodes[selected_node]['score'] = 0
            check_bfs[selected_node] = True
            queue_bfs.append(selected_node)
            queue_bfs.pop(0)

        list_new.append(G_new)
        for node in G_new.nodes:
            G_remaining.remove_node(node)
    return list_new
        #break

In [4]:
from scipy.optimize import minimize
from qiskit import QuantumCircuit, Aer
from qiskit.circuit import Parameter
import random

def mi_calculation_square(G, P, weightG, backend):
    appearance = [False for i in range(0, len(G.edges))]
    HP = []
    obj = {'cost': [], 'penalty': []}
    node_set = list(G.nodes())
    n = len(node_set)
    edge_to_index = {}
    idx = 0
    for ed in G.edges():
        i = ed[0]
        j = ed[1]
        edge_to_index[(i,j)] = idx
        edge_to_index[idx] = (i,j)
        idx = idx + 1
    
    for ed in G.edges:
        ed_idx = edge_to_index[(ed[0], ed[1])]
        obj['cost'].append([ed_idx])
        HP.append(set([ed_idx]))
        
    node_set = list(G.nodes())
    for i in range(0, n):
        s = []
        for j in range(0, n):
            if i == j:
                continue
            for k in range(j+1, n):
                if k == i:
                    continue
                for q in range(j+1,n):
                    if q == i or q == k:
                        continue
                    for permu in permu_list:
                        mapping = {permu[0]: node_set[i], permu[1]: node_set[j], permu[2]: node_set[k], permu[3]: node_set[q]}
                        check = True
                        motif = []
                        for ed in P.edges():
                            if not G.has_edge(mapping[ed[0]], mapping[ed[1]]):
                                check = False
                                break
                            else:
                                motif.append(edge_to_index[(mapping[ed[0]], mapping[ed[1]])])
                            
                        if check:
                            s.append(set(motif))
                            for ed_mo in motif:
                                appearance[ed_mo] = True
                
    ebunch = []
    for ed in G.edges:
        idx = edge_to_index[ed]
        if not appearance[idx]:
            ebunch.append(ed)
    G.remove_edges_from(ebunch)
    
    HP = []
    HP_cons2 = []
    obj = {'cost': [], 'penalty': [], 'constraint_2': []}
    node_set = list(G.nodes())
    n = len(node_set)
    edge_to_index = {}
    idx = 0
    for ed in G.edges():
        i = ed[0]
        j = ed[1]
        edge_to_index[(i,j)] = idx
        edge_to_index[idx] = (i,j)
        idx = idx + 1
    
    check_cost = []
    for i in range(0, n):
        for j in range(0, n):
            if j == i:
                continue
            for k in range(0, n):
                if k == i or k == j:
                    continue
                for q in range(0, n):
                    if q == i or q == k or q == j:
                        continue
                    for permu in permu_list:
                        mapping = {permu[0]: node_set[i], permu[1]: node_set[j], permu[2]: node_set[k], permu[3]: node_set[q]}
                        check = True
                        for ed in P.edges():
                            if not G.has_edge(mapping[ed[0]], mapping[ed[1]]):
                                check = False
                                break
                        if check:
                            s = []
                            for ed in P.edges():
                                s.append(edge_to_index[(mapping[ed[0]], mapping[ed[1]])])
                            if set(s) not in check_cost:
                                check_cost.append(set(s))
                                obj['cost'].append(s)
    if len(obj['cost']) == 0:
        return 0, [], [], -1
    for ed in G.edges:
        ed_idx = edge_to_index[(ed[0], ed[1])]
        HP.append(set([ed_idx]))
    
    node_set = list(G.nodes())
    for i in range(0, n):
        s_cons1 = []
        for j in range(0, n):
            if i == j:
                continue
            s_cons2 = []
            for k in range(0, n):
                if k == i or k == j:
                    continue
                for q in range(0,n):
                    if q == i or q == j or q == k:
                        continue
                    for permu in permu_list:
                        mapping = {permu[0]: node_set[i], permu[1]: node_set[j], permu[2]: node_set[k], permu[3]: node_set[q]}
                        check = True
                        motif = []
                        for ed in P.edges():
                            if not G.has_edge(mapping[ed[0]], mapping[ed[1]]):
                                check = False
                                break
                            else:
                                motif.append(edge_to_index[(mapping[ed[0]], mapping[ed[1]])])
                        
                        if check:
                            if set(motif) not in s_cons1:
                                s_cons1.append(set(motif))
                            if set(motif) not in s_cons2:
                                s_cons2.append(set(motif))
                            break
                
                    
            if G.has_edge(node_set[i],node_set[j]):
                obj['constraint_2'].append((edge_to_index[(node_set[i],node_set[j])], s_cons2))
                for idx_0 in range(len(s_cons2)):
                    it_0 = tuple(s_cons2[idx_0])
                    HP_cons2.append([-1, set(it_0 + tuple([edge_to_index[(node_set[i],node_set[j])]]))])
                    for idx_1 in range(idx_0 + 1, len(s_cons2)):
                        it_1 = tuple(s_cons2[idx_1])
                        if it_0 != it_1:
                            candidate = set(it_0+it_1)
                            HP_cons2.append([1, candidate])
                
        if len(s_cons1) > 1:
            ss = []
            for ele in s_cons1:
                ss.append(tuple(ele))
            s_cons1 = ss
            obj['penalty'].append(s_cons1)
            for idx_0 in range(len(s_cons1)):
                for idx_1 in range(idx_0+1, len(s_cons1)):
                    it_0 = s_cons1[idx_0]
                    it_1 = s_cons1[idx_1]
                    if it_0 != it_1:
                        candidate = set(it_0+it_1)
                        #if candidate not in HP:
                        HP.append(candidate)

    if len(obj['cost']) == 0:
        return 0, [], [], -1
    start = time.time()
    expectation = get_expectation_test(G, HP, HP_cons2, obj, backend)
    res_ = minimize(expectation,
                   [0.1, 0.1],
                   method='COBYLA')

    qc_res = create_qaoa_circ(G, res_.x, HP, HP_cons2)

    counts = backend.run(qc_res, seed_simulator=10, shots = 500000).result().get_counts()
    minn = 1000000
    removed = []
    saved = []
    for k in counts.keys():
        reverse_str = ''
        for b in k:
            reverse_str = b + reverse_str
        maxcut = maxcut_obj(reverse_str, obj)
        if maxcut % 100 != 0 or maxcut > 0:
            continue
        minn = min(minn, maxcut)
        if minn == maxcut:
            #print(obj)
            removed = []
            saved = []
            for comb in obj['cost']:
                summ = 0
                for b in comb:
                    summ = summ + int(reverse_str[b])
                if summ == len(comb):
                    new_motif = []
                    for b in comb:
                        removed.append(edge_to_index[b][0])
                        removed.append(edge_to_index[b][1])
                        new_motif.append((edge_to_index[b][0], edge_to_index[b][1]))
                    saved.append(new_motif)
    if minn >= 1000000:
        minn = 0
    return minn, list(set(removed)), saved, time.time() - start, counts, obj, qc_res, HP, HP_cons2, expectation, edge_to_index
    

In [6]:
def maxcut_obj(solution, obj):
    """Given a bit string as a solution, this function returns
    the number of edges shared between the two partitions
    of the graph.
    Args:
        solution: (str) solution bit string
        graph: networkx graph
    Returns:
        obj: (float) Objective
    """
    res = 0
    for comp in obj['cost']:
        s = 1
        for it in comp:
            s = s * int(solution[it])
        res = res + 100*s
    #print("Maxcut:",comp)
    for comp in obj['penalty']:
        s = 0
        for subcomp in comp:
            ss = 1
            for it in subcomp:
                ss = ss * int(solution[it])
            s += ss
        w = 49
        res = res - w*(1 - s)*(1 - s) - w*s*s + w

    for comp in obj['constraint_2']:
        s = 0
        init_element = comp[0]
        for subcomp in comp[1]:
            ss = 1
            for it in subcomp:
                ss = ss * int(solution[it])
            s += ss
        w = 49
        res = res - w*(int(solution[init_element]) - s)*(int(solution[init_element]) - s)
    #print(res)
    return -res


def compute_expectation(counts, graph, obj):
    """Computes expectation value based on measurement results
    Args:
        counts: (dict) key as bit string, val as count
        graph: networkx graph
    Returns:
        avg: float
             expectation value
    """
    avg = 0
    sum_count = 0
    for bit_string, count in counts.items():
        reverse_str = ''
        for b in bit_string:
            reverse_str = b + reverse_str
        obj_ = maxcut_obj(reverse_str, obj)
        avg += obj_ * count
        sum_count += count
    #print(avg, sum_count)
    return avg/sum_count


# We will also bring the different circuit components that
# build the qaoa circuit under a single function
def create_qaoa_circ(graph, theta, HP, HP_cons2):
    """Creates a parametrized qaoa circuit
    Args:
        graph: networkx graph
        theta: (list) unitary parameters
    Returns:
        (QuantumCircuit) qiskit circuit
    """
    nqubits = len(graph.edges())
    n_layers = len(theta)//2  # number of alternating unitaries
    beta = theta[:n_layers]
    gamma = theta[n_layers:]

    qc = QuantumCircuit(nqubits)

    # initial_state
    qc.h(range(nqubits))
    for layer_index in range(n_layers):
        # problem unitary
        for comp in HP:
            lstcomp = list(comp)
            if len(lstcomp) == 3:
                first_sign = 1
            else:
                first_sign = -1
            for mix in range(1, 1<<(len(lstcomp))):
                newlst = []
                for idx in range(0, len(lstcomp)):
                    if (mix & (1<<idx)) != 0:
                        newlst.append(lstcomp[idx])
                if (len(newlst) % 2 != 0):
                    sign = -first_sign
                else:
                    sign = first_sign
                for it in range(0, len(newlst)-1):
                    qc.cx(newlst[it], newlst[it+1])
                qc.rz(2 * sign * gamma[layer_index], newlst[-1])
                for it in range(len(newlst)-1, 0, -1):
                    qc.cx(newlst[it-1], newlst[it])
        for comp in HP_cons2:
            lstcomp = list(comp[1])
            first_sign = -comp[0]
            for mix in range(1, 1<<(len(lstcomp))):
                newlst = []
                for idx in range(0, len(lstcomp)):
                    if (mix & (1<<idx)) != 0:
                        newlst.append(lstcomp[idx])
                if (len(newlst) % 2 != 0):
                    sign = -first_sign
                else:
                    sign = first_sign
                for it in range(0, len(newlst)-1):
                    qc.cx(newlst[it], newlst[it+1])
                qc.rz(2 * sign * gamma[layer_index], newlst[-1])
                for it in range(len(newlst)-1, 0, -1):
                    qc.cx(newlst[it-1], newlst[it])
        # mixer unitary
        for qubit in range(nqubits):
            qc.rx(2 * beta[layer_index], qubit)
    #print('End circuit')
    qc.measure_all()
    return qc


# Finally we write a function that executes the circuit
# on the chosen backend
def get_expectation(graph, HP, obj, shots=512):
    """Runs parametrized circuit
    Args:
        graph: networkx graph
    """
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots

    def execute_circ(theta):
        qc = create_qaoa_circ(graph, theta, HP)
        counts = backend.run(qc, seed_simulator=10,
                             shots=10000).result().get_counts()
        return compute_expectation(counts, graph, obj)

    return execute_circ

def get_expectation_test(graph, HP, HP_cons2, obj, backend):
    """Runs parametrized circuit
    Args:
        graph: networkx graph
    """

    def execute_circ(theta):
        qc = create_qaoa_circ(graph, theta, HP, HP_cons2)
        counts = backend.run(qc, seed_simulator=10,
                             shots=10000).result().get_counts()
        return compute_expectation(counts, graph, obj)

    return execute_circ

In [ ]:
import time
import threading

permu_list = list(itertools.permutations([0,1,2,3]))
final_res = []
P = nx.DiGraph()
P.add_nodes_from([0, 1, 2, 3])
P.add_edges_from([(0, 2), (0, 3), (1, 2), (1,3)])
P[0][2]['weight'] = 0
P[0][3]['weight'] = 1
P[1][2]['weight'] = 1
P[1][3]['weight'] = 0

disease_list = ['Alzheimers', 'Parkinsons', 'Huntingtons', 'ALS', 'MND']
score_list = ['0.0']

for string_OoA in ["OR"]:
    for disease in disease_list:
        for score in score_list:
            temp_res = {'string_OoA':string_OoA, 'disease':disease, 'score':score, 'edge_size': -1, 'nums_motifs':0, 'running_time': -1, 'saved_motifs': []}
            edge_lst = []
            file_name = 'dataset/real/' + string_OoA + "_minGDA_" + score + "_" + disease + ".txt"
            weightG = {}
            with open(file_name) as f:
                while (True):
                    s = f.readline()
                    if s == '':
                        break

                    lst = list(s[:-1].split(" "))
                    #print(lst)
                    edge_lst.append((int(lst[0]), int(lst[1])))
                    weightG[(int(lst[0]), int(lst[1]))] = int(lst[2])
            G = nx.DiGraph()
            G.add_edges_from(edge_lst)

            G_new = G.copy()
            res = 0
            total_run_time = 0
            total_saved_motifs = []
            while True:
                check_terminal = True
                list_new = graph_partitioning(G_new, max_node = max(G.nodes)+1, cluster_limit = 20)
                print(len(list_new), len(G_new.nodes))

                list_backend = []
                for i in range(len(list_new)):
                    backend = Aer.get_backend('aer_simulator')
                    backend.shots = 1024
                    list_backend.append(backend)

                start = time.time()
                for i in range(len(list_new)):
                    p = mi_calculation_square(list_new[i], P, weightG, list_backend[0])
                    res_partition, removed_nodes, saved_motifs, run_time = p[0], p[1], p[2], p[3]
                    G_new.remove_nodes_from(removed_nodes)
                    if len(removed_nodes) != 0:
                        check_terminal = False
                    res += res_partition
                    total_saved_motifs = total_saved_motifs + saved_motifs
                    if run_time != -1:
                        total_run_time += run_time
                if check_terminal:
                    break
            temp_res['nums_motifs'] = -res/100
            temp_res['running_time'] = total_run_time
            temp_res['saved_motifs'] = total_saved_motifs
            temp_res['edge_size'] = len(edge_lst)
            final_res.append(temp_res)

In [16]:
import pickle
dataset_type = 'real'
motif_type = 'bifan'
results_file_name = 'dataset/'+dataset_type+'/'+motif_type+'_results.pkl'
with open(results_file_name, 'wb') as f:
    pickle.dump(final_res, f)